# Tutorial: Introduction to Unsupervised Learning with a Focus on PCA

Tutorial to the class [Introduction to Unsupervised Learning with a Focus on PCA](7_unsupervised_learning_pca.ipynb).

<div class="alert alert-block alert-info">
    <b>Tutorial Objectives</b>
    
- Apply Principal Component Analysis (PCA) to climate data to analyze patterns of variability
- (Combine PCA reduction/$k$-means clustering to Ordinary Least Squares (OLS) to predict climate variables)
- (Use cross-validation to regularize the OLS with the number of retained Empirical Orthogonal Functions (EOFs) or clusters).
</div>

## Dataset presentation

- Input:
  - [Geopotential height](https://en.wikipedia.org/wiki/Geopotential_height) at 500hPa
    - Domain: North Atlantic
    - Spatial resolution: $0.5° \times 0.625°$
    - Time resolution: monthly
    - Period: 1980-2018
    - Units: m
    - Source: [MERRA-2 reanalysis](https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/)
- Target:
  - Onshore wind capacity factors
    - Domain: Metropolitan France
    - Spatial resolution: regional mean
    - Time resolution: monthly
    - Period: 2014-2018
    - Units:
    - Source: [RTE](https://opendata.reseaux-energies.fr/)

In [ ]:
# Import modules
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Data directory
DATA_DIR = Path('data')

# Filename to geopotential height at 500hPa from MERRA-2 reanalysis
filename = 'merra2_analyze_height_500_month_19800101-20190101.nc'

> ***Question***
> - Read the geopotential height data using `xarray.load_dataset` and print it.

In [ ]:
# Import xarray

# Read dataset with xarray


# Select geopotential height variable


> ***Question***
> - Compute the mean and the variance of the geopotential height with the `mean` and `var` methods.
> - Plot the mean with the `plot` method.
> - Do a filled-contour plot of the variance with the `plot.contourf` method.

In [ ]:
# Compute the mean

# Plot mean per grid point with pcolormesh


# Compute deviations from the mean

# Compute the variance

# Plot variance per grid point with contourf


> ***Question***
> - Plot the variance of the geopotential height on the map. To do so, import `cartopy.crs` and use it to define the `projection` argument of `plt.subplot`, and use the `coastlines` and the `gridlines` method of the axes.

In [ ]:
# Import Cartopy's coordinate reference system

# Plot variance with costlines and gridlines


> ***Question***
> - Scale the geopotential-height deviations to account for variations in the area represented by each grid point.
> - Plot the variance of the scaled geopotential height.

In [ ]:
# Get grid-point area coefficients

# Scale height by coefficient

# Plot scaled variance with costlines and gridlines


> ***Question***
> - Estimate the covariance matrix of the scaled geopotential height using the `stack` method of data arrays.

In [ ]:
# Get input matrix


# Estimate covariance matrix


> ***Question***
> - Compute EOFs and corresponding variances using `np.linalg.eigh`.

In [ ]:
# Compute eigenvalues and eigenvectors


> ***Question***
> - Sort the EOFs and corresponding variances by decreasing variances.
> - Plot the fraction of variance "explained" by the leading 20 EOFs.
> - Interpret your results.

In [ ]:
# Sort eigenvalues and eigenvectors by decreasing eigenvalues


# Convert eigenvalues to explained variance percentage

# Plot first 20 explained variances


Answer: 

> ***Question***
> - Plot the leading EOF on a map.
> - To what physical phenomenon could this pattern be associated to?

In [ ]:
# Convert first eigenvector to data array

# Unstack latitudes and longitudes

# Plot first eigenvector


Answer: 

> ***Question***
> - Compute the principal component associated with the leading EOF.
> - Compare its variance to the corresponding eigenvalue and explain your result.
> - Plot this principal component.
> - Confirm or reconsider you previous answer on the physical phenomenon that could be associated to the leading EOF.

In [ ]:
# Compute first principal component

# Compare variance of first principal component to corresponding eigenvalue

# Plot first principal component


Answer: 

Answer: 

> ***Question (optional)***
> - Use the `scipy.signal.welch` to estimate the Power Spectral Density (PSD) of the leading principal component and plot it.
> - Confirm or reconsider you previous answer on the physical phenomenon that could be associated to the leading EOF.

In [ ]:
# Import signal module from Scipy

# Get PSD from Welch estimator

# Plot and give period at maximum in title


> ***Question***
> - Compute the seasonal cycle of the geopotential height (averages over all years of the same month of the year for each month) with the `groupby` of data arrays.
> - Plot all 12 months. You can use the `col` option of the `plot` method of data arrays.
> - Also plot the variance of the seasonal cycle on a map.
> - Confirm or reconsider you previous answer on the physical phenomenon that could be associated to the leading EOF.

In [ ]:
# Compute monthly seasonal cycle

# Get seasonal cycle

# Plot each month of seasonal cycle


# Plot variance of seasonal cycle


Answer: 

> ***Question***
> - Compute seasonal anomalies (deviations from the seasonal cycle) of the geopotential height with `groupby`.
> - Plot the variance of the seasonal anomalies on a map.
> - How does it compare to the variance of the data with the seasonal cycle?

In [ ]:
# Get seasonal anomalies

# Plot variance of the seasonal anomalies on a map


Answer:

> ***Question***
> - Estimate the covariance matrix of the seasonal anomalies.
> - Compute the EOFs and corresponding variances.
> - Plot the explained variances associated with the EOFs together with the cumulative sum of the explained variances.
> - What is the minimum number of EOFs that one needs to keep to explain at least 90% of the variance.

In [ ]:
# Get input matrix for seasonal anomalies


# Estimate covariance matrix


In [ ]:
# Compute eigenvalues and eigenvectors


In [ ]:
# Sort eigenvalues and eigenvectors by decreasing eigenvalues


# Convert eigenvalues to explained variance percentage


# Plot first 20 explained variances


Answer:

> ***Question***
> - Plot the leading 4 EOFs and principal components.
> - Can you associate these patterns to known climate phenomena?

In [ ]:
# Plot leading EOFs and principal components

    
# Compute the leading principal components

# Plot the leading principal component


Answer: 

> ***Question***
> - Reconstruct the inputs from the leading 4 EOFs only.
> - Compare the original time series at a few arbitrary locations to the corresponding reconstructed time series.
> - Plot the variance of the reconstruction on a map.
> - Same question but keeping more EOFs
> - Interpret your results in terms of filtering.

In [ ]:
# Reconstruct the data from first 4 principal components


# Compare original time series and reconstruction at arbitrary grid points


# Convert reconstruction to data array


# Unstack latitudes and longitudes


# Plot variance of reconstruction on a map


Answer: 

> ***Question (optional)***
> - Design a linear model that best predicts present (not future) wind capacity factors in `data/reseaux_energies_capacityfactor_wind-onshore.csv` using geopotential-height principal components as inputs. To do, use cross-validation to regularize based on the number of leading principal components retained.

> ***Question (optional)***
> - Use $k$-means clustering with `sklearn.cluster.KMeans` to detect "atmospheric regimes" from the geopotential-height data and compare the result to the EOFs obtained above.
> - Design a linear model as above but based on clusters rather than EOFs.

***
## Credit

[//]: # "This notebook is part of [E4C Interdisciplinary Center - Education](https://gitlab.in2p3.fr/energy4climate/public/education)."
Contributors include Bruno Deremble and Alexis Tantet.
Several slides and images are taken from the very good [Scikit-learn course](https://inria.github.io/scikit-learn-mooc/).

<br>

<div style="display: flex; height: 70px">
    
<img alt="Logo LMD" src="images/logos/logo_lmd.jpg" style="display: inline-block"/>

<img alt="Logo IPSL" src="images/logos/logo_ipsl.png" style="display: inline-block"/>

<img alt="Logo E4C" src="images/logos/logo_e4c_final.png" style="display: inline-block"/>

<img alt="Logo EP" src="images/logos/logo_ep.png" style="display: inline-block"/>

<img alt="Logo SU" src="images/logos/logo_su.png" style="display: inline-block"/>

<img alt="Logo ENS" src="images/logos/logo_ens.jpg" style="display: inline-block"/>

<img alt="Logo CNRS" src="images/logos/logo_cnrs.png" style="display: inline-block"/>
    
</div>

<hr>

<div style="display: flex">
    <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0; margin-right: 10px" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a>
    <br>This work is licensed under a &nbsp; <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.
</div>